In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
from sklearn.preprocessing import StandardScaler
import seaborn as sb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, accuracy_score, confusion_matrix, roc_auc_score, roc_curve
import statsmodels.api as sm
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

In [6]:
df_tom = pd.read_csv("rf_prediction_latest.csv", sep=',')
df_tom

,Unnamed: 0,index,region_name,period_end,total_homes_sold,average_homes_sold,total_homes_sold_with_price_drops,average_homes_sold_with_price_drops,percent_homes_sold_with_price_drops,median_sale_price,...,spring,summer,fall,region_type_county,us10yr_yield,headlineCPI,consumer_conf,unemployment_rate,classified,y_pred
0,661588,692584,"Zanesville, OH metro area",2022-07-17,18.0,18.0,3.0,3.0,0.166667,129900.0,...,0,1,0,0,2.9152,9.1,98.4,3.6,1,1
1,376371,1597903,"McCreary County, KY",2022-07-17,1.0,1.0,0.0,0.0,0.000000,400000.0,...,0,1,0,1,2.9152,9.1,98.4,3.6,0,1
2,374017,2043465,"Mathews County, VA",2022-07-17,3.0,3.0,1.0,1.0,0.333333,279950.0,...,0,1,0,1,2.9152,9.1,98.4,3.6,1,1
3,374306,2350736,"Maui County, HI",2022-07-17,38.0,38.0,9.0,9.0,0.236842,827500.0,...,0,1,0,1,2.9152,9.1,98.4,3.6,2,2
4,374595,2254851,"Maury County, TN",2022-07-17,44.0,44.0,13.0,13.0,0.295455,371332.5,...,0,1,0,1,2.9152,9.1,98.4,3.6,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2905,322854,173055,"Lanier County, GA",2018-03-04,1.0,1.0,0.0,0.0,0.000000,31530.0,...,1,0,0,1,2.8643,2.2,130.0,4.1,0,0
2906,233212,138871,"Guadalupe County, NM",2017-10-29,2.0,2.0,0.0,0.0,0.000000,48416.0,...,0,0,1,1,2.4064,2.2,120.6,4.3,0,0
2907,573364,2292785,"Stonewall County, TX",2017-08-13,1.0,1.0,0.0,0.0,0.000000,25000.0,...,0,1,0,1,2.1888,1.7,120.0,4.3,0,0
2908,492348,221490,"Quitman County, GA",2017-06-25,1.0,1.0,0.0,0.0,0.000000,164500.0,...,0,1,0,1,2.1423,1.9,117.6,4.4,1,1


In [10]:
df_tom = df_tom[['region_name', 'y_pred']]
df_tom[['city', 'state']]=df_tom['region_name'].str.split(', ',  expand=True)
df_tom['state']=df_tom['state'].str.replace(' metro area','')
df_tom['state']=df_tom['state'].str.strip()
df_tom['city']=df_tom['city'].str.replace(' County','')
df_tom['city']=df_tom['city'].str.strip()
df_tom = df_tom[['state', 'city', 'y_pred']]
df_tom

C:\Users\User\anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,state,city,y_pred
0,OH,Zanesville,1
1,KY,McCreary,1
2,VA,Mathews,1
3,HI,Maui,2
4,TN,Maury,1
...,...,...,...
2905,GA,Lanier,0
2906,NM,Guadalupe,0
2907,TX,Stonewall,0
2908,GA,Quitman,1


In [11]:
df_jes = pd.read_csv("housing_logistic_regression_df_02.csv", sep=',')
df_jes

,region_id,state,city,house_price,days_on_market_cutoff,season,probability_exceeding_cutoff
0,10140,WA,Aberdeen,250000,15,spring,43%
1,10140,WA,Aberdeen,250000,15,summer,43%
2,10140,WA,Aberdeen,250000,15,fall,43%
3,10140,WA,Aberdeen,250000,15,winter,43%
4,10140,WA,Aberdeen,350000,15,spring,40%
...,...,...,...,...,...,...,...
76091,49780,OH,Zanesville,850000,60,winter,28%
76092,49780,OH,Zanesville,950000,60,spring,25%
76093,49780,OH,Zanesville,950000,60,summer,25%
76094,49780,OH,Zanesville,950000,60,fall,25%


In [22]:
df_final = df_jes.merge(df_tom, on=['state', 'city'] )
df_final = df_final.dropna(axis=0)
df_final

,region_id,state,city,house_price,days_on_market_cutoff,season,probability_exceeding_cutoff,y_pred
0,10140,WA,Aberdeen,250000,15,spring,43%,1
1,10140,WA,Aberdeen,250000,15,summer,43%,1
2,10140,WA,Aberdeen,250000,15,fall,43%,1
3,10140,WA,Aberdeen,250000,15,winter,43%,1
4,10140,WA,Aberdeen,350000,15,spring,40%,1
...,...,...,...,...,...,...,...,...
89723,49780,OH,Zanesville,850000,60,winter,28%,1
89724,49780,OH,Zanesville,950000,60,spring,25%,1
89725,49780,OH,Zanesville,950000,60,summer,25%,1
89726,49780,OH,Zanesville,950000,60,fall,25%,1


In [18]:
df_final.to_csv("affordability_and_logistic_regres.csv", sep=",", index = False)